# Testing Queries

Note: For now, we are lucky that the imports work, because we are using relative imports, and at one point, it will cause errors.

## IMPORTS

In [1]:
import os
import sys
import random
import requests
import argparse
from typing import List, Set

import numpy as np
import pandas as pd

In [2]:
from db.yagodb import YagoDB
from db.constants.main import YAGO_ALL_ENTITY_COUNT, YAGO_FACTS_ENTITY_COUNT
from db.functions.entity import get_random_entities_query

In [3]:
from kg.query import get_triples_multiple_subjects_query, get_description_multiple_entities_query, query_kg, get_triples_from_response

In [4]:
from utils.constants import YAGO_ENTITY_STORE_DB_PATH, YAGO_PREFIXES_PATH, YAGO_ENDPOINT_URL
from utils.prefix import get_prefixes, get_url_from_prefix_and_id
from utils.random_walk2 import SPARQL_COLUMNS_DICT, RandomWalk2

### CONSTANTS

### FUNCTIONS

### Experiment Single Walks

In [5]:
yago_db = YagoDB(YAGO_ENTITY_STORE_DB_PATH)

In [6]:
random_walk_2 = RandomWalk2(yago_db)

In [7]:
entities_df = random_walk_2.random_walk_description_batch(num_of_entities=1000, depth=3)

In [8]:
entities_df

,entity0,description0,predicate1,entity1,description1,predicate2,entity2,description2
0,http://yago-knowledge.org/resource/Alsophila_D...,species of fern in the class Equisetopsida,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://schema.org/Taxon,"group of one or more organism(s), which a taxo...",http://www.w3.org/2002/07/owl#disjointWith,http://schema.org/CreativeWork,artistic creation
1,http://yago-knowledge.org/resource/Hosztót,NaN,None,None,NaN,None,None,NaN
2,http://yago-knowledge.org/resource/Roodhaar_Q2...,NaN,http://yago-knowledge.org/resource/appearsIn,http://yago-knowledge.org/resource/Storm__u002...,comic book series,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Comic_book_...,group of published comic stories typically sha...
3,http://yago-knowledge.org/resource/Cinemart_Q1...,NaN,http://schema.org/location,http://yago-knowledge.org/resource/Netherlands,country in Northwestern Europe with territorie...,http://schema.org/memberOf,http://yago-knowledge.org/resource/Internation...,Non-governmental sports organisation (founded ...
4,http://yago-knowledge.org/resource/Alavala_Bra...,NaN,http://schema.org/location,http://yago-knowledge.org/resource/India,country in South Asia,http://yago-knowledge.org/resource/replaces,http://yago-knowledge.org/resource/Nandgaon_State,Indian Princely State during the British Raj &...
...,...,...,...,...,...,...,...,...
995,http://yago-knowledge.org/resource/Tyc_648-317...,high proper-motion star in the constellation C...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/High_Proper...,total proper motion >= 50 mas/yr,http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Star,astronomical object consisting of a luminous s...
996,http://yago-knowledge.org/resource/Q109001960,encyclopedia article published in Church Encyc...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://yago-knowledge.org/resource/Encyclopedi...,"part of encyclopedia, sometimes considered as ...",http://www.w3.org/2000/01/rdf-schema#subClassOf,http://yago-knowledge.org/resource/Lemma_Q3055347,definition in a reference work (dictionary or ...
997,http://yago-knowledge.org/resource/Knapphenne_...,mountain in Italy,http://schema.org/location,http://yago-knowledge.org/resource/Italy,country in Southern Europe,http://yago-knowledge.org/resource/capital,http://yago-knowledge.org/resource/Rome,capital and largest city of Italy
998,http://yago-knowledge.org/resource/Sidomulyo_Q...,"village in Silo District, Jember Regency, East...",http://schema.org/location,http://yago-knowledge.org/resource/Indonesia,country in Southeast Asia and Oceania,http://yago-knowledge.org/resource/capital,http://yago-knowledge.org/resource/Jakarta,capital city and smallest province of Indonesia


In [13]:
entities_df["description0"].isna().sum()

265

In [14]:
entities_df["entity1"].to_csv("entity1.csv", index=False)